# Techniques 2: Ergodic Jump Diagnostics

## Overview

**Ergodic Jump** is a specialized diagnostic technique for uncovering hidden structure in ergodic (fully mixed, high-entropy) signals.

## What is Ergodicity?

An **ergodic system** is one where:
- Time averages equal ensemble averages
- The system explores all possible states
- No long-term memory or structure
- Maximum entropy (fully mixed)

Examples: White noise, ideal gas, random walk

## The Ergodic Jump Technique

### Concept

Like "uncoiling a quantum yo-yo":
1. **Inject** a targeted harmonic into the ergodic signal
2. **Induce** non-ergodic "stickiness" at resonant frequencies
3. **Peel** away the resonant component
4. **Extract** the hidden structure

### Why It Works

If the signal has hidden structure:
- Injection at resonant frequency amplifies it
- Non-ergodic behavior emerges
- Structure becomes visible in the residual

If truly ergodic:
- Injection has no lasting effect
- Signal remains fully mixed
- No structure emerges

## Use Cases

- **Signal analysis**: Find hidden periodicities
- **Quality control**: Detect non-random patterns
- **Cryptography**: Test randomness
- **Physics**: Identify quantum vs classical behavior

## Architecture Context

**Layer 4: Processors** (`workbench.processors.ergodic`)
- Specialized diagnostic technique
- Uses Layer 1 primitives (FFT, signal processing)

---

In [ ]:
import sys
import os
# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath('../..'))

import numpy as np
import matplotlib.pyplot as plt

# Layer 4: Ergodic diagnostics
from workbench.processors.ergodic import ErgodicJump

print('✓ Imports successful')
print('\nErgodicJump provides:')
print('  - Ergodicity diagnosis')
print('  - Filament injection')
print('  - Resfrac/Hurst shift detection')

## 1. Testing True Ergodic Signal

First, test on white noise (truly ergodic).

In [ ]:
print('Testing on Ergodic Signal (White Noise)')
print('=' * 70)

# Create truly ergodic signal (white noise)
np.random.seed(42)
ergodic_signal = np.random.randn(512)

# Initialize ErgodicJump
jump = ErgodicJump(
    injection_freq=1/np.sqrt(5),  # Golden ratio related
    amp=0.15
)

print(f'Injection frequency: {jump.injection_freq:.4f}')
print(f'Injection amplitude: {jump.amp}')

# Execute diagnostic
result = jump.execute(ergodic_signal)

print(f'\nResults:')
print(f'  Is ergodic: {result.is_ergodic}')
print(f'  Stickiness: {result.stickiness:.6f}')
print(f'  Structure detected: {result.structure_detected}')

if result.is_ergodic:
    print('\n✓ Correctly identified as ergodic (no hidden structure)')
else:
    print('\n✗ False positive: detected structure in noise')

## 2. Testing Non-Ergodic Signal

Test on signal with hidden periodic structure.

In [ ]:
print('Testing on Non-Ergodic Signal (Hidden Structure)')
print('=' * 70)

# Create signal with hidden periodic structure
t = np.linspace(0, 10, 512)
hidden_freq = 1/np.sqrt(5)  # Match injection frequency
non_ergodic_signal = (
    0.3 * np.sin(2*np.pi*hidden_freq*t) +  # Hidden structure
    0.7 * np.random.randn(len(t))  # Noise
)

# Execute diagnostic
result = jump.execute(non_ergodic_signal)

print(f'Results:')
print(f'  Is ergodic: {result.is_ergodic}')
print(f'  Stickiness: {result.stickiness:.6f}')
print(f'  Structure detected: {result.structure_detected}')

if not result.is_ergodic and result.structure_detected:
    print('\n✓ Correctly detected hidden periodic structure')
    print(f'  Resonant frequency: {result.resonant_freq:.4f}')
    print(f'  (True frequency: {hidden_freq:.4f})')
else:
    print('\n✗ Failed to detect hidden structure')

## 3. Visualizing the Jump

Compare before/after injection and extracted structure.

In [ ]:
print('Visualizing Ergodic Jump Process')
print('=' * 70)

# Create signal with subtle structure
t = np.linspace(0, 20, 1024)
signal = (
    0.2 * np.sin(2*np.pi*0.447*t) +  # Subtle structure
    0.8 * np.random.randn(len(t))  # Dominant noise
)

# Execute and get detailed results
jump = ErgodicJump(injection_freq=0.447, amp=0.2)
result = jump.execute(signal)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original signal
axes[0, 0].plot(t, signal, linewidth=0.5, alpha=0.7)
axes[0, 0].set_title('Original Signal (Noisy)', fontweight='bold')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].grid(True, alpha=0.3)

# After injection
if hasattr(result, 'injected_signal'):
    axes[0, 1].plot(t, result.injected_signal, linewidth=0.5, alpha=0.7, color='orange')
    axes[0, 1].set_title('After Harmonic Injection', fontweight='bold')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('Amplitude')
    axes[0, 1].grid(True, alpha=0.3)

# Extracted structure
if hasattr(result, 'extracted_structure'):
    axes[1, 0].plot(t, result.extracted_structure, linewidth=1.5, color='green')
    axes[1, 0].set_title('Extracted Structure', fontweight='bold')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('Amplitude')
    axes[1, 0].grid(True, alpha=0.3)

# Power spectrum
freqs = np.fft.rfftfreq(len(signal), t[1] - t[0])
power = np.abs(np.fft.rfft(signal))**2
axes[1, 1].semilogy(freqs, power, linewidth=1, alpha=0.7)
axes[1, 1].axvline(jump.injection_freq, color='r', linestyle='--', 
                   label=f'Injection: {jump.injection_freq:.3f}')
axes[1, 1].set_title('Power Spectrum', fontweight='bold')
axes[1, 1].set_xlabel('Frequency')
axes[1, 1].set_ylabel('Power')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\n✓ Visualization shows injection → extraction process')

## 4. Frequency Scanning

Scan multiple frequencies to find resonances.

In [ ]:
print('Frequency Scanning for Resonances')
print('=' * 70)

# Create signal with multiple hidden frequencies
t = np.linspace(0, 20, 1024)
multi_freq_signal = (
    0.3 * np.sin(2*np.pi*0.3*t) +
    0.2 * np.sin(2*np.pi*0.7*t) +
    0.5 * np.random.randn(len(t))
)

# Scan frequencies
test_freqs = np.linspace(0.1, 1.0, 20)
stickiness_values = []

for freq in test_freqs:
    jump_test = ErgodicJump(injection_freq=freq, amp=0.15)
    result = jump_test.execute(multi_freq_signal)
    stickiness_values.append(result.stickiness)

# Find peaks
stickiness_array = np.array(stickiness_values)
peaks = []
for i in range(1, len(stickiness_array) - 1):
    if stickiness_array[i] > stickiness_array[i-1] and \
       stickiness_array[i] > stickiness_array[i+1]:
        peaks.append((test_freqs[i], stickiness_array[i]))

print(f'\nScanned {len(test_freqs)} frequencies')
print(f'Found {len(peaks)} resonant peaks:')
for freq, stick in peaks:
    print(f'  Frequency: {freq:.3f}, Stickiness: {stick:.6f}')

# Visualize scan
plt.figure(figsize=(12, 5))
plt.plot(test_freqs, stickiness_values, 'o-', linewidth=2, markersize=6)
for freq, stick in peaks:
    plt.axvline(freq, color='r', linestyle='--', alpha=0.5)
plt.axhline(0.3, 0.7, color='orange', linestyle=':', alpha=0.5, 
            label='True frequencies')
plt.xlabel('Injection Frequency', fontsize=12)
plt.ylabel('Stickiness', fontsize=12)
plt.title('Frequency Scan: Resonance Detection', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print('\n✓ Frequency scan identified resonant peaks')

## 5. Practical Applications

Real-world use cases for ergodic jump diagnostics.

In [ ]:
print('Practical Applications')
print('=' * 70)

print('\n1. RANDOMNESS TESTING')
print('   Use: Verify cryptographic random number generators')
print('   Method: Should show no stickiness at any frequency')
rng_test = np.random.randn(1024)
jump_rng = ErgodicJump(injection_freq=0.5, amp=0.1)
result_rng = jump_rng.execute(rng_test)
print(f'   Result: Ergodic={result_rng.is_ergodic}, '
      f'Stickiness={result_rng.stickiness:.6f}')
print(f'   ✓ RNG is {"good" if result_rng.is_ergodic else "suspicious"}')

print('\n2. HIDDEN PERIODICITY DETECTION')
print('   Use: Find subtle cycles in noisy time series')
print('   Method: Scan frequencies, look for resonance peaks')
print('   Example: Market cycles, biological rhythms, sensor drift')

print('\n3. SIGNAL QUALITY ASSESSMENT')
print('   Use: Distinguish signal from noise')
print('   Method: High stickiness = structure present')
print('   Example: Sensor validation, data quality checks')

print('\n4. QUANTUM VS CLASSICAL BEHAVIOR')
print('   Use: Identify quantum coherence')
print('   Method: Quantum systems show non-ergodic signatures')
print('   Example: Quantum computing error detection')

print('\n✓ Ergodic jump is a versatile diagnostic tool')

## Summary

### Key Concepts

**Ergodicity**: System fully explores state space, no memory
**Non-Ergodicity**: Hidden structure, long-range correlations
**Harmonic Injection**: Probe signal with targeted frequency
**Stickiness**: Measure of non-ergodic behavior
**Structure Extraction**: Isolate hidden periodic components

### The Technique

1. Inject harmonic at test frequency
2. Measure stickiness (persistence of injection)
3. High stickiness → resonance → hidden structure
4. Extract structure via filtering/peeling

### When to Use

- **Unknown periodicities**: Find hidden cycles
- **Randomness testing**: Verify true randomness
- **Quality control**: Detect non-random patterns
- **Signal analysis**: Separate structure from noise
- **Physics research**: Identify quantum behavior

### Parameters

**injection_freq**: Frequency to test (scan for unknowns)
**amp**: Injection amplitude (0.1-0.2 typical)
**threshold**: Stickiness threshold for detection

### Interpretation

**is_ergodic=True**: No hidden structure at test frequency
**is_ergodic=False**: Structure detected, resonance found
**stickiness**: Strength of non-ergodic behavior
**structure_detected**: Significant periodic component found

### Limitations

- Requires sufficient signal length
- May miss very weak signals
- Frequency resolution limited by signal length
- Best for periodic/quasi-periodic structure

### Next Steps

- **Utilities 1-3**: Foundational tools
- **Techniques 1**: Other core processors
- **Your data**: Apply to real signals

**Architecture**: `workbench/processors/ergodic.py` (Layer 4)

---

*"Like uncoiling a quantum yo-yo" - inject, resonate, extract*